In [1]:
import os
import netCDF4

# stage_in packages
from unity_sds_client.resources.collection import Collection

# stage_out packages
from datetime import datetime, timezone
from unity_sds_client.resources.dataset import Dataset
from unity_sds_client.resources.data_file import DataFile

In [ ]:

input_stac_collection_file = 'test' # type: stage-in
output_stac_catalog_dir    = 'process_results/'                    # type: stage-out

# pre-process variables
output_collection="L1B_processed"
crid = "001"
temp_directory = "/tmp"
sensor = "EMIT"


# Import Files from STAC Item Collection

Load filenames from the stage_in STAC item collection file

In [4]:
inp_collection = Collection.from_stac(input_stac_collection_file)
data_filenames = inp_collection.data_locations()

data_filenames

['/home/mcduffie/Devel/unity/unity-example-application/test/stage_in/./SNDR.SS1330.CHIRP.20160822T0005.m06.g001.L1_AQ.std.v02_48.G.200425095850.nc',
 '/home/mcduffie/Devel/unity/unity-example-application/test/stage_in/./SNDR.SS1330.CHIRP.20160822T0011.m06.g002.L1_AQ.std.v02_48.G.200425095901.nc']

# Preprocess Command


In [ ]:
emit_directory = os.path.dirname(run_config['inputs']['raw_dataset'])
obs_base_name = raw_file.replace('RAD','OBS')


# This is the code we're actually interested in
emit.nc_to_envi(raw_file,
                    output,
                    temp_directory,
                    obs_file = obs_base_name,
                    export_loc = True,
                    crid = str(run_config['inputs']['crid']))

for dataset in glob.glob(output+"/SISTER*.bin"):
        generate_metadata(dataset.replace('.bin','.hdr'),
                                  output)

#Update crid
for file in glob.glob(output+"/SISTER*"):
    os.rename(file,file.replace('CRID',
                                    str(run_config['inputs']['crid'])))

rdn_file =  glob.glob(output+"/*%s.bin" % run_config['inputs']['crid'])[0]
generate_quicklook(rdn_file,output)

if os.path.exists(run_config_json):
    shutil.copyfile(run_config_json,
                output+'/%s.runconfig.json' % os.path.basename(rdn_file)[:-4])

#added this check because i don't think this script is creating a run.log... not sure where this comes from
if os.path.exists('run.log'):
    shutil.copyfile('run.log',
                output+'/%s.log' % os.path.basename(rdn_file)[:-4])
# done being "interested in"

# Create stage-out item catalog

In [9]:
# Create a collection
out_collection  = Collection("SNDR13CHRP1AQCal_rebin")
    
# Create a Dataset for the collection
dataset = Dataset(
    name=os.path.splitext(summary_table_filename)[0], 
    collection_id=out_collection.collection_id, 
    start_time=datetime.utcnow().replace(tzinfo=timezone.utc).isoformat(), 
    end_time=datetime.utcnow().replace(tzinfo=timezone.utc).isoformat(),
    creation_time=datetime.utcnow().replace(tzinfo=timezone.utc).isoformat(),
)

# Add output file(s) to the dataset
dataset.add_data_file(DataFile("data", summary_table_filename))

# Add the dataset to the collection
#out_collection.add_dataset(dataset)
out_collection._datasets.append(dataset)

Collection.to_stac(out_collection, output_stac_catalog_dir)